First, we'll upload a HDF5 file that was generated from within the SLEAP GUI. This can be created by opening a tracked project file (`.slp`) and going to **File** -> **Export Analysis HDF5...**

Note that you can also upload the file by navigating the sidebar on the left side of the page in Colab.

In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec

import numpy as np
import os
import shutil
import cv2
from tqdm import trange

#import glob2

import h5py
#import hdf5storage
import csv



Autosaving every 180 seconds


In [20]:
#################################################
######## LOAD VIDEO AND SHOW FRAME ##############
#################################################

video_name = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort2/2020_08_01_10_18_35_455807.avi'
original_vid = cv2.VideoCapture(video_name)

#
start=0
original_vid.set(cv2.CAP_PROP_POS_FRAMES, start)
ret, frame = original_vid.read()
print ('frame.', frame.shape)
z_min = frame.min(0)
z_max = frame.max(0)
rgb = 1
plt.imshow(frame[:,:,:])
plt.show()
    
    

frame. (1024, 1280, 3)


In [22]:
import csv

traces =[]
with open('/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_9/2020-3-9_12_14_22_815059_compressed/video.mp4.analysis.csv', mode='r') as infile:
    reader = csv.reader(infile)
    with open('coors_new.csv', mode='w') as outfile:
        writer = csv.writer(outfile)
        for row in reader:
            traces.append(row)
            
traces.pop(0)

print (traces[0])

['track_0', '0', '', '', '', '', '', '', '', '', '', '', '908.9131469726562', '164.5895538330078', '900.7461547851562', '148.29443359375', '880.7421264648438', '136.3643798828125', '852.9115600585938', '128.3528289794922', '', '', '', '', '', '', '', '', '', '']


In [44]:
#########################################################################
######################### MAKE VIDEOS ###################################
#########################################################################
import cv2
import matplotlib

#          pup1     pup2    female  male
colors_4= ['blue', 'red', 'cyan', 'orange','green', 'white','yellow','pink']

video_name = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_9/2020-3-9_12_14_22_815059_compressed/2020-3-9_12_14_22_815059_compressed.avi'
original_vid = cv2.VideoCapture(video_name)


# SELECT VIDEO SIZE
size_vid = np.array([1280,1024])
scale = 1
dot_size = 8//scale

# SET START AND END TIMES
start = 56500
end = start+300
#end = len(valid_frame_idxs)
#end = 3000
print ("START: ", start, "   END: ", end)

# SELECT VIDEOS OUT
#out_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/'
fname_out = video_name[:-4]+"_corrected_"+str(start)+"_"+str(end)+".mp4"
fourcc = cv2.VideoWriter_fourcc('M','P','E','G')
video_out = cv2.VideoWriter(fname_out,fourcc, 25, (size_vid[0]//scale,size_vid[1]//scale), True)


original_vid.set(cv2.CAP_PROP_POS_FRAMES, start)

font = cv2.FONT_HERSHEY_PLAIN

ctr_sleap = 0
for n in trange(start,end, 1):
    ret, frame = original_vid.read()
    #print (n, frame.shape)
    cv2.putText(frame, "Frame: "+str(n), (50, 100), font, 4, (0, 255, 0), 5)
    frame = frame[::scale, ::scale]
    
    # LOAD ALL LAbELS AT THIS TRACES
    while True:
        temp = traces[ctr_sleap]
        #print (temp)
        frame_id = int(temp[1])
        track_id = int(temp[0].replace('track_',''))
        
        # TRACK TEXT
        track_text = True
        #print (locs1)
        
        # EXIT WHEN HITTING NEXT FRAME INFO
        if frame_id>n:
            break
        
        # 
        locs1 = np.array(temp[2:])  # LOAD x,y locations

        y_array = locs1[::2]
        #print (x_array)
        x_array = locs1[1::2]
        
        for k in range(len(x_array)):
            x = x_array[k]
            y = y_array[k]
            
            if x=='' or y=='':
                continue
            else:
                x=int(float(x))//scale
                y=int(float(y))//scale

                if track_text:
                    cv2.putText(frame, "tr #: "+str(track_id), (y-220,x+40), font, 3, (0, 255, 0), 5)
                    track_text=False
                
                frame[x-dot_size:x+dot_size,y-dot_size:y+dot_size]= (np.float32(
                    matplotlib.colors.to_rgb(colors_4[track_id%8]))*255.).astype('uint8')
                #print (colors_4[k])
                #frame[y-dot_size:y+dot_size,x-dot_size:x+dot_size]= (np.float32(
                #    matplotlib.colors.to_rgb(colors_4[z//14]))*255.).astype('uint8')
                
        ctr_sleap+=1

    #print ("")
    video_out.write(frame)

    if n==2771:
        fig=plt.figure(figsize=(10,10))
        plt.imshow(frame)
        plt.savefig('/home/cat/frame.png', dpi=600)
        plt.close()
    
    #print ("")

video_out.release()
original_vid.release()
#cv2.destroyAllWindows()

START:  56500    END:  56800


100%|██████████| 300/300 [01:13<00:00,  4.08it/s]


In [5]:
import cv2
import matplotlib
matplotlib.use('Agg')


#          pup1     pup2    female  male
colors_4= ['orange','green', 'blue', 'red', 'cyan','white','yellow','pink']

video_name = '/media/cat/4TBSSD/dan/march_2/sleap_talmo/2020-3-9_12_14_22_815059_compressed.avi'
original_vid = cv2.VideoCapture(video_name)


# SELECT VIDEO SIZE
size_vid = np.array([1280,1024])
scale = 1
dot_size = 8//scale

# SET START AND END TIMES
# start = 0
# end = 299+1
#end = len(valid_frame_idxs)
#end = 3000
start=225
end=start+100

print ("START: ", start, "   END: ", end)

# SELECT VIDEOS OUT
#out_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/'
fname_out = video_name[:-4]+"_corrected_"+str(start)+"_"+str(end)+".mp4"
fourcc = cv2.VideoWriter_fourcc('M','P','E','G')
video_out = cv2.VideoWriter(fname_out,fourcc, 25, (size_vid[0]//scale,size_vid[1]//scale), True)


original_vid.set(cv2.CAP_PROP_POS_FRAMES, start)

font = cv2.FONT_HERSHEY_PLAIN

ctr_sleap = 0
ctr_show = 0

fig=plt.figure(figsize=(20,20))

for n in trange(start,end, 1):
    ret, frame = original_vid.read()
    #print (n, frame.shape)
    cv2.putText(frame, str(n), (50, 100), font, 5, (255, 255, 0), 5)
    frame = frame[::scale, ::scale]
    
    # LOAD ALL LAbELS AT THIS TRACES
    while True:
        temp = traces[ctr_sleap]
        #print (temp)
        frame_id = int(temp[1])
        track_id = int(temp[0].replace('track_',''))
        
#         if frame_id !=n:
#             ctr_sleap+=1

#             continue
        
        # TRACK TEXT
        track_text = True
        #print (locs1)
        
        # EXIT WHEN HITTING NEXT FRAME INFO
        if frame_id>n:
            break
        
        if frame_id<n:
            ctr_sleap+=1
            continue
        
        # 
        locs1 = np.array(temp[2:])  # LOAD x,y locations

        y_array = locs1[::2]
        #print (x_array)
        x_array = locs1[1::2]
        
        for k in range(len(x_array)):
            x = x_array[k]
            y = y_array[k]
            
            if x=='' or y=='':
                continue
            else:
                x=int(float(x))//scale
                y=int(float(y))//scale

                if track_text:
                    cv2.putText(frame, str(track_id), (y,x), font, 5, (255, 255, 0), 5)
                    track_text=False
                
                frame[x-dot_size:x+dot_size,y-dot_size:y+dot_size]= (np.float32(
                    matplotlib.colors.to_rgb(colors_4[track_id%8]))*255.).astype('uint8')

        
        ctr_sleap+=1

    ax=plt.subplot(2,5,ctr_show+1)
    plt.imshow(frame)
    plt.title(str(n))
    
    ctr_show+=1
fig.savefig('/home/cat/fig2.png',dpi=300)   

NameError: name 'valid_frame_idxs' is not defined